**NER For Medical**

In [ ]:
## install required library
!pip install transformers --quiet
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00


In [ ]:
# import the library
import torch
import numpy as np
import pandas as pd

In [ ]:
# load the data from huggigface
from datasets import load_dataset
data = load_dataset("lcampillos/ctebmsp")

README.md:   0%|          | 0.00/5.25k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

(…)-00000-of-00001-dd47cca0d1defaa4.parquet:   0%|          | 0.00/607k [00:00<?, ?B/s]

(…)-00000-of-00001-9ce2de2af3726115.parquet:   0%|          | 0.00/224k [00:00<?, ?B/s]

(…)-00000-of-00001-9933eb5ee9c7d6f5.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4549 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4505 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 12554
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 4549
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 4505
    })
})

In [ ]:
data['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
pd.DataFrame(data['train'][:])[['tokens', 'ner_tags']].iloc[0]

,0
tokens,"[Efecto, de, la, suplementación, con, ácido, f..."
ner_tags,"[O, O, O, B-PROC, O, B-CHEM, I-CHEM, O, O, O, ..."


In [ ]:
tags = set([tag for tags in data['train']['ner_tags'] for tag in tags])
tag2id = {tag: id for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
tag2id

{'O': 0,
 'B-CHEM': 1,
 'I-CHEM': 2,
 'I-PROC': 3,
 'I-DISO': 4,
 'I-ANAT': 5,
 'B-DISO': 6,
 'B-PROC': 7,
 'B-ANAT': 8}

In [ ]:
data['train'][0]

{'tokens': ['Efecto',
  'de',
  'la',
  'suplementación',
  'con',
  'ácido',
  'fólico',
  'sobre',
  'los',
  'niveles',
  'de',
  'homocisteína',
  'total',
  'en',
  'pacientes',
  'en',
  'hemodiálisis'],
 'ner_tags': ['O',
  'O',
  'O',
  'B-PROC',
  'O',
  'B-CHEM',
  'I-CHEM',
  'O',
  'O',
  'O',
  'O',
  'B-CHEM',
  'O',
  'O',
  'O',
  'O',
  'B-PROC']}

In [ ]:
def create_tag_names(batch):
  tag_name = {'ner_tags_int': [tag2id[idx] for idx in batch['ner_tags']]}
  return tag_name

In [ ]:
data = data.map(create_tag_names)

Map:   0%|          | 0/12554 [00:00<?, ? examples/s]

Map:   0%|          | 0/4549 [00:00<?, ? examples/s]

Map:   0%|          | 0/4505 [00:00<?, ? examples/s]

In [ ]:
data['train'][0]['ner_tags_int']

[0, 0, 0, 7, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 7]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', do_lower_case=False)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = data['train'][0]['tokens']
inputs = tokenizer(inputs, is_split_into_words=True)
print(inputs.tokens())
print(len(inputs.tokens()))

['[CLS]', 'Efec', '##to', 'de', 'la', 'suplement', '##ación', 'con', 'ácido', 'f', '##ólico', 'sobre', 'los', 'niveles', 'de', 'homo', '##ciste', '##ína', 'total', 'en', 'pacientes', 'en', 'hemo', '##di', '##álisis', '[SEP]']
26


In [ ]:
print(data['train'][0]['tokens'])
print(data['train'][0]['ner_tags_int'])

['Efecto', 'de', 'la', 'suplementación', 'con', 'ácido', 'fólico', 'sobre', 'los', 'niveles', 'de', 'homocisteína', 'total', 'en', 'pacientes', 'en', 'hemodiálisis']
[0, 0, 0, 7, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 7]


In [ ]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word=None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = labels[word_id]

      if label%2==1:
        label = label + 1
      new_labels.append(label)

  return new_labels

In [ ]:
labels = data['train'][0]['ner_tags_int']
word_ids = inputs.word_ids()
print(labels, word_ids)

[0, 0, 0, 7, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 7] [None, 0, 0, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 16, 16, None]


In [ ]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  all_labels = examples['ner_tags_int']

  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels

  return tokenized_inputs

In [ ]:
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)

Map:   0%|          | 0/12554 [00:00<?, ? examples/s]

Map:   0%|          | 0/4549 [00:00<?, ? examples/s]

Map:   0%|          | 0/4505 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 12554
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4549
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4505
    })
})

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch

{'input_ids': tensor([[    4, 20110,  1071,  1008,  1030, 13561,  1086,  1051, 10728,  1060,
         13765,  1269,  1065,  4618,  1008,  9477, 29317,  7900,  2318,  1036,
          5172,  1036, 20767,  1046,  3853,     5],
        [    4,  1198, 11693,  2872,  1186, 29317,  1955,  6125,  1058,  1049,
         20245,  1008,  3921,  5414,  1008,  2377,  1133,  3855,  6208, 24092,
          2194,  2227,  1009,     5,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0]]), 'labels': tensor([[-100,    0,    0,    0,    0,    7,    8,    0,    1,    2,    2,    0,
            0,    0,    0,    1,    2,    2, 

In [ ]:
!pip install seqeval
!pip install evaluate

import evaluate
metric = evaluate.load('seqeval')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=8958c7ceb37587f346d4b738ae0be946d1f7c1be763dbfef3a2c5cb0b3bad27f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
tags

{'B-ANAT',
 'B-CHEM',
 'B-DISO',
 'B-PROC',
 'I-ANAT',
 'I-CHEM',
 'I-DISO',
 'I-PROC',
 'O'}

In [ ]:
import numpy as np
label_names = list(tags)
def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
                                                    'dccuchile/bert-base-spanish-wwm-cased',
                                                    id2label=id2tag,
                                                    label2id=tag2id)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments("distilbert-finetuned-ner",
                         evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=3,
                         weight_decay=0.01)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer
trainer = Trainer(model=model,
                  args=args,
                  train_dataset = tokenized_datasets['train'],
                  eval_dataset = tokenized_datasets['validation'],
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.030400,0.288227,0.845592,0.862642,0.854032,0.942181
2,0.056600,0.248312,0.846681,0.876847,0.861500,0.943702
3,0.021600,0.288910,0.856190,0.874728,0.865360,0.945951


TrainOutput(global_step=4710, training_loss=0.0371449293351224, metrics={'train_runtime': 663.3326, 'train_samples_per_second': 56.777, 'train_steps_per_second': 7.101, 'total_flos': 1116933655264596.0, 'train_loss': 0.0371449293351224, 'epoch': 3.0})